## Data prep

In [234]:
import numpy as np
import pandas as pd
import scipy.special as spc
import scipy.stats as sts

macro = np.loadtxt(
    'C:/Users/Hrisi/Structural_estimation/StructEst_W18/ProblemSets/PS3/MacroSeries.txt', delimiter=",")


## A. First approach

In [168]:
def calculate_errors(x, a,r,m):
    k=x[:,1]
    w=x[:,2]
    z_0=m
    z=[]
    e=[]
    z.append(np.log(w[0]/(k[0]**a * (1-a))))
    e.append(z[0]-r*z_0-(1-r)*m)
    
    for t in range(1, 100):
        z.append(np.log(w[t]/(k[t]**a * (1-a))))
        e.append(z[t]-r*z[t-1]-(1-r)*m)
        
    return np.array(e)

In [178]:
def norm_pdf(xvals, mu, sigma):
    pdf_vals = np.maximum(sts.norm.pdf(xvals, mu, sigma), 1e-10)
    #pdf_vals    = np.maximum((1/(sigma * np.sqrt(2 * np.pi)) *
    #               np.exp( - (xvals - mu)**2 / (2 * sigma**2))), 1e-10)
    return pdf_vals

In [179]:
def log_lik_norm(x, mu, sigma):
    pdf_vals = norm_pdf(x, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

In [180]:
# Define the criterion function for the normal distribution:
def crit(params, args):
    a, r, m, sigma = params
    x = args
    e=calculate_errors(x, a, r, m)
    log_lik_val = log_lik_norm(e,0,sigma)
    neg_log_lik_val = (-log_lik_val)
    
    return neg_log_lik_val

In [230]:
# MLE optimization procedure for the Normal distribution
import scipy.optimize as opt

#Initial values
a_init=0.3
r_init=0.9
m_init=9
sigma_init=0.1
# Set arguments of the opt.minimize function
bnds = ((1e-10, 1-1e-10), (-0.99999999,0.99999999), (1e-10, None), (1e-10, None))
params_init = np.array([a_init, r_init, m_init, sigma_init])
mle_args = (macro)

# run the optimization procedure
results = opt.minimize(crit, params_init, args=(mle_args), bounds=bnds)

# report results
a_mle1, r_mle1, m_mle1, sigma_mle1 = results.x
print("a_mle1= ", a_mle1, "r_mle1= ", r_mle1, "m_mle1= ", m_mle1, "sigma_mle1= ", sigma_mle1)
crit([a_mle1, r_mle1, m_mle1, sigma_mle1], macro)

a_mle1=  0.49558713399 r_mle1=  0.691257415972 m_mle1=  8.9930388742 sigma_mle1=  0.0920069139467


-96.69516631224387

In [221]:
results

      fun: -82.359568106047405
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  65.194466  ,   27.30807722,    3.89916295,  337.86214573])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 40
      nit: 3
   status: 0
  success: True
        x: array([  0.42710611,   0.99999999,  10.01078266,   0.13292288])

In [184]:
results.hess_inv
OffDiagNeg = np.array([[1, -1, -1, -1], [-1, 1,-1,-1], [-1, -1,1,-1], [-1, -1,-1,1]])
vcv_mle = results.hess_inv * OffDiagNeg
print(results.hess_inv)
print('VCV(MLE) = ', vcv_mle)

<4x4 LbfgsInvHessProduct with dtype=float64>
VCV(MLE) =  [[  7.73667766e+03   6.29145878e+03  -7.27416310e+03   6.72237942e+03]
 [ -3.46131248e+03  -2.81471575e+03   3.25436671e+03  -3.00752823e+03]
 [ -1.08571684e+05  -8.82903831e+04   1.02081038e+05  -9.43376604e+04]
 [ -1.22398970e+02  -9.95353624e+01   1.15082287e+02  -1.06351968e+02]]


## B. Second approach

In [217]:
def calculate_errors2(x, a,r,m):
    k=x[:,1]
    rate=x[:,3]
    z_0=m
    z=[]
    e=[]
    z.append(np.log(rate[0]/(k[0]**(a-1)*a)))
    e.append(z[0]-r*z_0-(1-r)*m)
    
    for t in range(1, 100):
        z.append(np.log(rate[t]/(k[t]**(a-1)*a)))
        e.append(z[t]-r*z[t-1]-(1-r)*m)
        
    return np.array(e)

In [209]:
# Define the criterion function for the normal distribution:
def crit2(params, args):
    a, r, m, sigma = params
    x = args
    log_lik_val = log_lik_norm(calculate_errors2(x, a, r, m),0,sigma)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [215]:
# MLE optimization procedure for the Normal distribution
import scipy.optimize as opt

#Initial values
a_init=0.5
r_init=0.7
m_init=9
sigma_init=0.3

# Set arguments of the opt.minimize function
bnds = ((1e-10, 1-1e-10), (-1+1e-10,1-1e-10), (1e-10, None), (1e-10, None))
params_init = np.array([a_init, r_init, m_init, sigma_init])
mle_args = (macro)

# run the optimization procedure
results2 = opt.minimize(crit2, params_init, args=(mle_args), bounds=bnds)

# report results
a_mle2, r_mle2, m_mle2, sigma_mle2 = results2.x
print("a_mle2= ", a_mle2, "r_mle2= ", r_mle2, "m_mle2= ", m_mle2, "sigma_mle2= ", sigma_mle2)

a_mle2=  0.457484632888 r_mle2=  0.72051227185 m_mle2=  9.37086232624 sigma_mle2=  0.0919962303213


In [216]:
results2

      fun: -96.706908033083025
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.64321898e-04,   2.84217094e-06,  -1.70530257e-05,
        -2.13162821e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 205
      nit: 25
   status: 0
  success: True
        x: array([ 0.45748463,  0.72051227,  9.37086233,  0.09199623])

In [214]:
results2.hess_inv
OffDiagNeg = np.array([[1, -1, -1, -1], [-1, 1,-1,-1], [-1, -1,1,-1], [-1, -1,-1,1]])
vcv_mle = results2.hess_inv * OffDiagNeg
print(results.hess_inv)
print('VCV(MLE) = ', vcv_mle)

<4x4 LbfgsInvHessProduct with dtype=float64>
VCV(MLE) =  [[  9.11697713e+03   7.63471992e+03  -8.53899580e+03   8.17677792e+03]
 [ -5.53643589e+03  -4.63607481e+03   5.18513164e+03  -4.96545338e+03]
 [ -1.65425071e+05  -1.38529788e+05   1.54937581e+05  -1.48365393e+05]
 [ -1.77614672e+02  -1.48690162e+02   1.66289765e+02  -1.59290936e+02]]


## C. Inference

In [141]:
k = 7500000
z_lag = 10
z = np.log(1/(k**(a_mle2-1)*a_mle2))
e = z - r_mle2*z_lag + (1-r_mle2)*m_mle2
print(e)


6.68526230928


In [233]:
1-sts.norm.cdf(e,0,1)

1.1525558285541138e-11